# Imports

In [13]:
from PMP_importlib import measurement as ms
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from glob import glob

# Gather Hall data

First set the independent variables as categorical values.

In [2]:
datafolder = "../data/Hall_anneal/"

In [3]:
doping = CategoricalDtype(
    ['int', 'r96', 'r48', 'r24', 'r16'],
    ordered = True
)

anneal = CategoricalDtype(
    ['asdep'] + [f'{T}C' for T in range(300, 750, 50)],
    ordered = True,)

capping = CategoricalDtype(
    ['cap', 'nocap'],
    ordered = True,
)

Get all possible combinations

In [3]:
index = pd.MultiIndex.from_product(
    [doping.categories,
     capping.categories,
     anneal.categories],
    names = ['doping', 'capping', 'anneal'])
params = pd.DataFrame(index = index).reset_index()

For each combination, was I able to do a measurement?

If not, filter out.


In [4]:
def filter_true(f):
    # use this for pipelining with a filtering function
    def wrapped(df):
        return df[df.apply(f, axis = 1)]
    return wrapped

@filter_true
def measurement_valid(x):
    # I was able to measure all capped samples
    if x.capping == 'cap':
        return True
    # handle uncapped samples
    max_temp_nocap = {
        'int' : '300C',
        'r16' : '450C',
        'r24' : '400C',
        'r48' : '400C',
        'r96' : '350C',
    }
    T_max = max_temp_nocap[x.doping]
    # valid anneals include asdep up to maximal annealing temperature
    valid_anneal = {T for T in anneal.categories if T == 'asdep' or T <= T_max}
    return x.anneal in valid_anneal

For these valid measurements, get their filenames.

In [5]:
def set_column(colname):
    # again, only used for pipelining
    def decorator(f):
        def wrapped(df):
            new_df = df.copy()
            new_df[colname] = new_df.apply(f, axis = 1)
            return new_df
        return wrapped
    return decorator

@set_column('filename')
def create_filename(x):
    # derive relevant filename from parameters
    
    # special case : int, cap should be version 2
    if {x.doping, x.capping} == {'int', 'cap'}:
        return '_'.join([x.doping, x.capping, x.anneal, 'no2']) + '.xlsx'
    
    # otherwise default
    return '_'.join([x.doping, x.capping, x.anneal]) + '.xlsx'

In [6]:
long_names = {
        'Hall mobility'               : 'mu_H'  , # m^2 V s-1
        'Carrier concentration'       : 'n'     , # m^-3
        'Sheet carrier concentration' : 'n_sh'  , # m^-2
        'Hall coefficient'            : 'R_H'   , # m^3 C^-1
        'Sheet Hall coefficient'      : 'R_H_sh', # m^2 C^-1
        'Resistivity'                 : 'rho'   , # Ohm m
        'Sheet resistivity'           : 'rho_sh', # Ohm sq
        'Hall voltage'                : 'V_H'   , # V
        'Thickness'                   : 'd'     , # nm
    }

units = {
        'mu_H'  : 'm^2 V s-1',
        'n'     : 'm^-3',
        'n_sh'  : 'm^-2',
        'R_H'   : 'm^3 C^-1',
        'R_H_sh': 'm^2 C^-1',
        'rho'   : 'Ohm m',
        'rho_sh': 'Ohm sq',
        'V_H'   : 'V',
        'd'     : 'nm',
    }


def get_data(x):
    filepath = datafolder + x.filename
    data = ms.HallMeasurement(filepath).asDict()
    data = pd.Series(data).rename(long_names)
    return x.append(pd.Series(data))

In [22]:
import pickle
import os
def get_df(use_cache = True, cache = 'halldata'):
    if use_cache and os.path.exists(cache):
        df = pd.read_pickle(cache)
    else:
        df = params\
        .pipe(measurement_valid)\
        .pipe(create_filename)\
        .apply(get_data, axis = 1)\
        .astype('float', errors = 'ignore')
        df.to_pickle(cache)
    return df
        
    

In [24]:
if __name__ == "__main__":
    df = get_df(use_cache = True)
    import seaborn as sns

    sns.set_theme(
        context = 'paper',
        style = 'whitegrid',
        )
    plot_val = 'R_H'
    p = sns.lineplot(x = 'anneal', y = plot_val,
                    style = 'capping', hue = 'doping',
                    markers = True,
                    data = df)
    p.set(yscale = 'log')
    p.set(xlabel = 'Anneal temperature')
    p.set(ylabel = 'Hall resistivity $\\frac{m^3}{C}$')